# Question 1, 2

In [1]:
#imports
import nltk
from sklearn.model_selection import train_test_split

# custom functions
from helper import *

D:\Users\apoor\Miniconda3\envs\nlp\lib\site-packages\sklearn\utils\__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\apoor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
filename = "../books/alice_adventure.txt"
sentences = getSents(filename)

In [3]:
# lower case words, and remove punctuations.
normalizedSentences = lowerPunct(sentences)

# add Start stop symbols
finalSentences = addStartStop(normalizedSentences)

# divide the corpus into test and train
test, train = train_test_split(sentences,\
                               test_size=0.2,\
                               random_state=42)

----

# Question 3

In [4]:
# DEMO
se = [['<s>', 'i', 'am', 'not',  'dog', '</s>'],
      ['<s>', 'i', 'love', 'dog',  'food', '</s>'],
      ['<s>', 'dog', 'is', 'not', 'bad', '</s>']]

# get the counts of the required nGrams

nGramCount(se, 2)

({'<s> i': 2,
  'i am': 1,
  'am not': 1,
  'not dog': 1,
  'dog </s>': 1,
  'i love': 1,
  'love dog': 1,
  'dog food': 1,
  'food </s>': 1,
  '<s> dog': 1,
  'dog is': 1,
  'is not': 1,
  'not bad': 1,
  'bad </s>': 1},
 15)

In [5]:
# finding the MLEs for all the ngrams
N = 4
mles = []
for i in range(1, N + 1):
    nGrams, counts = nGramCount(finalSentences, i)
    mles.append(MLE(nGrams, counts))

In [6]:
# change the value of ngram to get it's MLE
ngram = 3
mles[ngram - 1]

{'<s> s adventures': 3.721207159602575e-05,
 's adventures in': 3.721207159602575e-05,
 'adventures in wonderland': 3.721207159602575e-05,
 'in wonderland lewis': 3.721207159602575e-05,
 'wonderland lewis carroll': 3.721207159602575e-05,
 'lewis carroll the': 3.721207159602575e-05,
 'carroll the millennium': 3.721207159602575e-05,
 'the millennium fulcrum': 3.721207159602575e-05,
 'millennium fulcrum edition': 3.721207159602575e-05,
 'fulcrum edition chapter': 3.721207159602575e-05,
 'edition chapter i': 3.721207159602575e-05,
 'chapter i </s>': 3.721207159602575e-05,
 '<s> down the': 3.721207159602575e-05,
 'down the alice': 3.721207159602575e-05,
 'the alice was': 3.721207159602575e-05,
 'alice was beginning': 7.44241431920515e-05,
 'was beginning to': 0.000148848286384103,
 'beginning to get': 3.721207159602575e-05,
 'to get very': 3.721207159602575e-05,
 'get very tired': 3.721207159602575e-05,
 'very tired of': 0.00011163621478807726,
 'tired of sitting': 3.721207159602575e-05,
 '

In [7]:
# Possible number of ngrams possible
poss_avai=[]
for i in range(1, N+1):
    nGrams, counts = nGramCount(finalSentences, i)
    poss_avai.append(possible_avail(nGrams))

In [8]:
# first val contains the possible no of ith ngrams
# second val is the number of unique ith ngrams
# where i > 1
s = ["unigrams", "bigrams", "trigrams", "quadgrams"]
for so in range(len(s)):
    print (s[so])
    print ("possible:", poss_avai[so][0])
    print ("present:", str(poss_avai[so][1]) + '\n')

unigrams
possible: 3111265
present: 2495

bigrams
possible: 99722503
present: 14123

trigrams
possible: 251922681
present: 22447

quadgrams
possible: 302125071
present: 24582



----

# Question 4

In [9]:
for i in range(1, N+1):
    print (Generator(mles[i-1], i))

TypeError: only integer scalar arrays can be converted to a scalar index